# Unlimited holdem poker bot

Для хакатона Сбербанка по написанию покерного бота.<br>
Правила : за столом играют 9 игроков 1500 фишек у каждого, 50 раундов.<br>
В качестве движка используется  pypokerengine

В своём решении я вдохновлялся [ссылка](https://github.com/Rayveni/outlace.github.io/blob/master/pelican/content/rlpart3.ipynb)



In [1]:
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate
from pypokerengine.players import BasePokerPlayer
import random
import pandas as pd
import numpy as np
import os
from random import randint
from standartbots import *
from pypokerengine.api.game import setup_config, start_poker

Базовый класс покерного бота 

In [44]:
class CorePlayer(BasePokerPlayer):
    def __init__(self):
        self.NB_SIMULATION = 450 # число симуляций длч Монте Карло для подсчёта WinRate
        #self.street_dict={'preflop':0,'flop':1/3,'turn':2/3,'river':1}
    def declare_action(self, valid_actions, hole_card, round_state):
        pass
    def receive_game_start_message(self, game_info):
        pass
    def receive_round_start_message(self, round_count, hole_card, seats):
        self.state_list=[]
        self.counter=0
        self.start_amount=sum((el['stack'] for el in seats))              
        self.nb_player = len(seats)
        self.start_stack=self.find_player(seats)['stack']
        self.total_stack=sum((el['stack'] for el in seats))

    def receive_street_start_message(self, street, round_state):
        self.state_list.append(['street',self.get_stats(round_state)])
    def receive_game_update_message(self, action, round_state):
        pass
    def receive_round_result_message(self, winners, hand_info, round_state):
        pass
    def get_stats(self,round_state,hole_card='smth'):
        alive_players=[el['stack'] for el in round_state['seats'] if el['state']== 'participating']
        self.alive_players=max(len(alive_players),1)
        self.pot=round_state['pot']['main']['amount']
        self.street=round_state['street']#self.street_dict[round_state['street']]
        
        current_pos=0
        for el in round_state['seats']:
            if el['uuid']==self.uuid:break
            current_pos=current_pos+1
             
        self.dealer_pos=current_pos-round_state['dealer_btn']
        try:
            self.max_stack=max(alive_players)
            self.min_stack=min([el for el in alive_players if el >0])
        except:
            pass
        self.small_blind_pos=round_state['small_blind_pos']-current_pos
        self.big_blind_pos=round_state['big_blind_pos']-current_pos
        self.current_stack=self.find_player(round_state['seats'])['stack'] 
        if hole_card!='smth': 
            if self.alive_players<2:
                self.win_rate=1
            else:
                self.win_rate=self.calc_win_rate(hole_card, round_state,self.NB_SIMULATION,self.alive_players)
                print(self.win_rate)
        else:
            self.win_rate='look_prev_state'
            
        features={'start_stack':self.start_stack/self.total_stack,
                  'current_stack':self.current_stack/self.total_stack,
                  'current_stack_min':self.current_stack/self.total_stack,
                  'current_stack_max':self.current_stack/self.total_stack,
                 'alive_players':self.alive_players/self.nb_player,
                  'pot':self.pot/self.total_stack,

                  'street':self.street,
                 'dealer_pos':self.dealer_pos/self.alive_players,
                 'max_stack':self.max_stack/self.total_stack,
                 'min_stack':self.min_stack/self.total_stack,
                 'blind_pos':self.small_blind_pos/self.alive_players,
                 'big_blind_pos':self.big_blind_pos/self.alive_players,
                 'win_rate':self.win_rate ,
                 'current_stack_abs':self.current_stack}
        return features
        
    def find_player(self,input_json):
        for el in input_json:
            if el['uuid']==self.uuid:return el

        
    def calc_win_rate(self,hole_card, round_state,nb_simulation,nb_player):
        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(nb_simulation,nb_player,
                                               hole_card=gen_cards(hole_card),
                                               community_card=gen_cards(community_card))
      
        return win_rate    

Класс бота на нейронке <br>
Переменная tournirs - костыль для обучения

In [45]:
class NeuralBot(CorePlayer):
    def __init__(self):
        super().__init__()
        super()
    def declare_action(self, valid_actions, hole_card, round_state):
       
        features=self.get_stats(round_state,hole_card)
        #print(valid_actions)
        #call_action_info = valid_actions[1]       
        #action, amount = call_action_info["action"], call_action_info["amount"]
        action, amount,n_action=make_move(valid_actions,features)
        self.state_list.append(['action',features,[n_action,action, amount]])         
        return action, amount
    
    def receive_round_result_message(self, winners, hand_info, round_state):
        self.state_list.append(['end',self.get_stats(round_state)])
        stack=self.find_player(round_state['seats'])['stack']
        reward=stack-self.start_stack
        #print('reward',reward)
        self.start_stack=stack
        global tournirs
        tournirs.append(self.state_list)

Функция **make_move** отвечает за ход<br>
Очередной костыль - **epsilon**, данная переменная отвечает как часто бот будет совершать случайные ходы, нужно в процессе обучения(иногда называют **exploration rate**)
Нужен на обучении, не нужен в бою.<br>
У бота в моей реализации возможно 5 действий:
* fold (сброс карт)
* call (подтверждение ставки)
* raise (1/3) увеличение ставки на 1/3 от возможной ставки
* raise  (2/3) увеличение ставки на 2/3 от возможной ставки
* all in

In [10]:
def preprocess_input(features):
    street=features['street']
    if street=='preflop':
        res=[0,0,0,0]
    elif street=='flop':
        res=[0,1,0,0]
    elif street=='turn':
        res=[0,0,1,0]    
    elif street=='river':
        res=[0,0,0,1]   
    else:
        print('error',features)
    
    return np.array(res+[features['start_stack'],features['current_stack'],
            features['current_stack_min'],features['current_stack_max'],
            features['alive_players'], features['pot'],
            features['dealer_pos'],features['max_stack'],
            features['min_stack'],features['blind_pos'],
            features['big_blind_pos'],features['win_rate']])

def make_move(valid_actions,features_dict):
    global epsilon
    raise_w={2:0,3:1/3,4:2/3,5:1}
    if len(valid_actions)>2:
        n=6
    else:
        n=2
        
    if (random.random() < epsilon):
        action = np.random.randint(0,6)#random_action(n)
    else:
        qval = model.predict(preprocess_input(features_dict).reshape(1,input_num), batch_size=1)[:n]
        action = (np.argmax(qval))
    return_action=action
    if action>1:
        k=raise_w[action]
        action=2
        a=valid_actions[action]['amount']['min']
        b=valid_actions[action]['amount']['max']
        amount=int(a+(b-a)*k)
    else:
        amount=valid_actions[action]['amount']
    return valid_actions[action]['action'],amount,return_action

def RunInt(VectorSize, Sum):
   x = [randint(0, Sum) for _ in range(1, VectorSize)]
   x.extend([0, Sum])
   x.sort()
   return [x[i+1] - x[i] for i in range(VectorSize)]


Инициализация игры.
Добавляем случайное число игроков суммарно равное 9
* CallPlayer
* RandomPlayer
* NeuralBot

In [5]:
def init_game(verbose=0,n_players=9):
    config = setup_config(max_round=50, initial_stack=1500, small_blind_amount=15)
    call_player_count,random_player_count,neural_player_count=RunInt(3,n_players-1)
    if call_player_count>0:
        for i in range(call_player_count):
            config.register_player(name="CallPlayer %s" % i, algorithm=CallPlayer())
    if random_player_count>0:
        for i in range(random_player_count):
            config.register_player(name="Random %s" % i, algorithm=RandomPlayer())            
    for i in range(neural_player_count+1):        
        config.register_player(name="NeuralBot %s" % i, algorithm=NeuralBot())
    game_result = start_poker(config, verbose=verbose)

#init_game(verbose=1)


In [6]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.models import model_from_json

Using TensorFlow backend.


Создаем нейронную сеть с двумя слоями

In [7]:
save_folder=r'C:\Users\Волочков\neural\holdem\save'
input_num=16
model = Sequential()
model.add(Dense(30, input_shape=(input_num,)))
model.add(Activation('relu'))
model.add(Dropout(0.2)) 

model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(6))#fold/call/raise 1/4...1 
model.add(Activation('linear')) #linear output so we can have range of real-valued outputs

rms = RMSprop()
model.compile(loss='mse', optimizer=rms)

model_json = model.to_json()
with open(os.path.join(save_folder,'model.json'), "w") as json_file:
    json_file.write(model_json)

загружаем веса модели (если есть расчётные)

In [63]:
model.load_weights("model299.h5")
print("Loaded model from disk")

Loaded model from disk


In [47]:
epsilon=-1 # 
tournirs=[]
init_game(1)

Started the round 1
Street "preflop" started. (community card = [])
"Random 1" declared "fold:0"
"Random 2" declared "raise:1043"
"Random 3" declared "raise:-1"
"Random 4" declared "raise:-1"
"Random 5" declared "fold:0"
0.19777777777777777
"NeuralBot 0" declared "raise:-1"
"CallPlayer 0" declared "call:1043"
"CallPlayer 1" declared "call:1043"
"Random 0" declared "raise:-1"
Street "flop" started. (community card = ['C5', 'SA', 'S3'])
"CallPlayer 1" declared "call:0"
"Random 2" declared "raise:160"
"CallPlayer 0" declared "call:160"
"CallPlayer 1" declared "call:160"
Street "turn" started. (community card = ['C5', 'SA', 'S3', 'S2'])
"CallPlayer 1" declared "call:0"
"Random 2" declared "fold:0"
"CallPlayer 0" declared "call:0"
Street "river" started. (community card = ['C5', 'SA', 'S3', 'S2', 'SK'])
"CallPlayer 1" declared "call:0"
"CallPlayer 0" declared "call:0"
"['CallPlayer 0']" won the round 1 (stack = {'CallPlayer 0': 3936, 'CallPlayer 1': 297, 'Random 0': 1470, 'Random 1': 1500, 

In [40]:
%%time
epochs = 300
gamma = 0.975
epsilon = 1
batchSize = 60
batch_percent=0.6
buffer = 200
replay = []
#stores tuples of (S, A, R, S')
h = 0
    
for epoch in range(epochs):
    tournirs=[]
    memory=[]
    new_round = init_game(0) #using the harder state initialization function
  
    X_train=[]
    y_train=[]
    for game in tournirs:
        action_nums=[i for i in range(len(game)) if game[i][0]=='action']
        if len(action_nums)>0:
            next_state_nums=[i+1 for i in action_nums]
            next_state_nums[-1] =len(game)-1     

            for i in range(len(action_nums)):
                state=game[action_nums[i]]
                new_state=game[next_state_nums[i]]
                action=state[2][0]
                memory.append(action)
                reward=new_state[1]['current_stack_abs']-state[1]['current_stack_abs']
                terminal_state=False
                if new_state[0]=='end': terminal_state=True
                new_state[1]['street']=state[1]['street']
                state=  preprocess_input(state[1])  
                new_state=  preprocess_input(new_state[1]) 
                new_state[-1]=state[-1]
                new_state[-2]=state[-2]
                #memory.append(state+ [action, reward]+ new_state)
                old_qval = model.predict(state.reshape(1,input_num), batch_size=1)

                newQ = model.predict(new_state.reshape(1,input_num), batch_size=1)
                maxQ = np.max(newQ)
                y = np.zeros((1,6))
                y[:] = old_qval[:]
                if terminal_state:
                    update = reward
                else:
                    update = (reward + (gamma * maxQ))
                y[0][action] = update

                X_train.append(state)
                y_train.append(y.reshape(6,))
                
    X_train = np.array(X_train)
    y_train = np.array(y_train)  

    if epoch >0:
        batch_index=np.random.choice(X_old.shape[0],int( X_old.shape[0]*batch_percent))
        X_train,X_old=np.vstack([X_train,X_old[batch_index]]),np.vstack([X_train,X_old])
        y_train,y_old=np.vstack([y_train,y_old[batch_index]]),np.vstack([y_train,y_old])
    else:
        #pass
        X_old=X_train
        y_old=y_train
        

    print("Game #: %s round" % (epoch,))
    df=pd.DataFrame(np.hstack([X_old,y_old]))
    df.to_csv(os.path.join(save_folder,'%s.csv'%epoch),sep=';',index=False)
    model.fit(X_train, y_train, epochs=100, verbose=0)
    
    model.save_weights('model%s.h5'%epoch)
    print('End epoch %s'% epoch)
    if epsilon > 0.1: #decrement epsilon over time
        epsilon -= (1/epochs)

Game #: 0 round
End epoch 0
Game #: 1 round
End epoch 1
Game #: 2 round
End epoch 2
Game #: 3 round
End epoch 3
Game #: 4 round
End epoch 4
Game #: 5 round
End epoch 5
Game #: 6 round
End epoch 6
Game #: 7 round
End epoch 7
Game #: 8 round
End epoch 8
Game #: 9 round
End epoch 9
Game #: 10 round
End epoch 10
Game #: 11 round
End epoch 11
Game #: 12 round
End epoch 12
Game #: 13 round
End epoch 13
Game #: 14 round
End epoch 14
Game #: 15 round
End epoch 15
Game #: 16 round
End epoch 16
Game #: 17 round
End epoch 17
Game #: 18 round
End epoch 18
Game #: 19 round
End epoch 19
Game #: 20 round
End epoch 20
Game #: 21 round
End epoch 21
Game #: 22 round
End epoch 22
Game #: 23 round
End epoch 23
Game #: 24 round
End epoch 24
Game #: 25 round
End epoch 25
Game #: 26 round
End epoch 26
Game #: 27 round
End epoch 27
Game #: 28 round
End epoch 28
Game #: 29 round
End epoch 29
Game #: 30 round
End epoch 30
Game #: 31 round
End epoch 31
Game #: 32 round
End epoch 32
Game #: 33 round
End epoch 33


Game #: 263 round
End epoch 263
Game #: 264 round
End epoch 264
Game #: 265 round
End epoch 265
Game #: 266 round
End epoch 266
Game #: 267 round
End epoch 267
Game #: 268 round
End epoch 268
Game #: 269 round
End epoch 269
Game #: 270 round
End epoch 270
Game #: 271 round
End epoch 271
Game #: 272 round
End epoch 272
Game #: 273 round
End epoch 273
Game #: 274 round
End epoch 274
Game #: 275 round
End epoch 275
Game #: 276 round
End epoch 276
Game #: 277 round
End epoch 277
Game #: 278 round
End epoch 278
Game #: 279 round
End epoch 279
Game #: 280 round
End epoch 280
Game #: 281 round
End epoch 281
Game #: 282 round
End epoch 282
Game #: 283 round
End epoch 283
Game #: 284 round
End epoch 284
Game #: 285 round
End epoch 285
Game #: 286 round
End epoch 286
Game #: 287 round
End epoch 287
Game #: 288 round
End epoch 288
Game #: 289 round
End epoch 289
Game #: 290 round
End epoch 290
Game #: 291 round
End epoch 291
Game #: 292 round
End epoch 292
Game #: 293 round
End epoch 293
Game #: 

Тесты (не смотреть)

In [ ]:
#df=pd.read_csv(r'C:\82.csv',sep=';')
#y_old=df.iloc[:,-6:].values
#X_old=df.iloc[:,:-6].values

In [ ]:
model_json = model.to_json()
with open(os.path.join(save_folder,'model%s.json'%epoch), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(r'\neural\holdem\save','model%s.h5'%epoch))
#print("Saved model to disk")

In [ ]:
# later...

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [66]:
f={'start_stack': 0.11148272017837235, 'current_stack': 0.11148272017837235, 'current_stack_min': 0.11148272017837235, 'current_stack_max': 0.11148272017837235, 'alive_players': 9.0, 'pot': 0.0033444816053511705, 'street': 'river', 'dealer_pos': 0.3333333333333333, 'max_stack': 0.11148272017837235, 'min_stack': 0.10925306577480491, 'blind_pos': -0.2222222222222222, 'big_blind_pos': -0.1111111111111111,
   'win_rate':1, 'current_stack_abs': 1500}
v=[{'action': 'fold', 'amount': 0}, {'action': 'call', 'amount': 30}, {'action': 'raise', 'amount': {'min': 45, 'max': 1500}}]
t=model.predict(preprocess_input(f).reshape(1,input_num), batch_size=1)[0]
print(t,np.argmax(t))

[  -9.9848814    88.01134491   72.21170807   63.95601654   39.41465759
  520.42297363] 5
